In [ ]:
parent_dir = "C:/Users/Davide/git/gitdf/EnergyCommunity.jl/run_cloud" #git/gitdf/EnergyCommunity.jl/run_cloud"
saveiter_dir = "$parent_dir/results_paper/iter"
saveenum_dir = "$parent_dir/results_paper/enum"

In [ ]:
using EnergyCommunity
using FileIO
using HiGHS, Plots
using JuMP
using Gurobi
using Games
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings
using YAML
using CSV

## Compare ENUM_MODE

In [ ]:
EC_size_list_enum = []
dict_enums = Dict()

for filename in readdir(saveenum_dir)
    if endswith(filename, ".jld2")
        size_enum = parse(Int, replace(filename, "enum_simulations_results_"=>"", ".jld2"=>""))
        push!(EC_size_list_enum, size_enum)
        dict_enums[size_enum] = load("$saveenum_dir/enum_simulations_results_$size_enum.jld2")
    end
end

### ENUM_MODE: Create reward redistribution of enum modes

In [ ]:
df_reward_enum = DataFrame()

for EC_size in EC_size_list_enum
    df_reward_temp = deepcopy(dict_enums[EC_size]["df_reward_enum"])
    reward_list = setdiff(names(df_reward_temp), ["user_set"])
    rename!(df_reward_temp, reward_list .=> reward_list .* "_$EC_size")
    if nrow(df_reward_enum) == 0
        df_reward_enum = df_reward_temp
    else
        df_reward_enum = outerjoin(df_reward_enum, df_reward_temp, on=:user_set, makeunique=true)
    end
end

# sort by user set
sort!(df_reward_enum, :user_set)

df_reward_enum

### ENUM_MODE: Create comparison of computational time

In [ ]:
df_time_enum = DataFrame()

for EC_size in EC_size_list_enum
    df_time_temp = DataFrame(dict_enums[EC_size]["df_time_enum"])  # dict_time_enum

    df_time_temp[!, 2:end] ./= 3600  # change units to hours

    df_time_temp[!, "EC size"] = [EC_size]
    df_time_temp[!, "title"] = [L"Time [h]"]
    df_time_temp = df_time_temp[!, ["EC size"; "title"; names(df_time_temp)[1:end-2]]]

    if nrow(df_time_enum) == 0
        df_time_enum = df_time_temp
    else
        df_time_enum = vcat(df_time_enum, df_time_temp)
    end
end

df_time_enum

## Compare ITER_MODE

In [ ]:
df_run_simulations_iter = CSV.read("$saveiter_dir/options_backup.csv", DataFrame)

In [ ]:
dict_iter = Dict()
n_iter = 0

while isfile("$saveiter_dir/iter_simulations_results_$(n_iter+1).jld2")
    n_iter += 1
    dict_iter[n_iter] = load("$saveiter_dir/iter_simulations_results_$n_iter.jld2")
end

#### ITER_MODE: Comparison of reward distribution

In [ ]:
df_reward_iter = DataFrame()

largest_user_set = []

for id_run in 1:n_iter
    df_reward_temp = deepcopy(dict_iter[id_run]["df_reward_iter"])
    reward_list = setdiff(names(df_reward_temp), ["user_set"])
    rename!(df_reward_temp, reward_list .=> reward_list .* "_$(nrow(df_reward_temp)-1)")
    if nrow(df_reward_iter) == 0
        df_reward_iter = df_reward_temp
    else
        df_reward_iter = outerjoin(df_reward_iter, df_reward_temp, on=:user_set, makeunique=true)
    end

    if length(largest_user_set) < nrow(df_reward_iter)
        largest_user_set = df_reward_temp[!, "user_set"]
    end
end

# sort by user set
sort!(df_reward_iter, :user_set, by=x -> findfirst(x .== largest_user_set))
CSV.write("reward.csv", df_reward_iter)
df_reward_iter

#### ITER_MODE: Create comparison of computational time

In [ ]:
df_time_iter = DataFrame()

for id_run in 1:n_iter
    df_time_temp = deepcopy(dict_iter[id_run]["df_time_iter"])

    df_time_temp[!, 3:end] ./= 3600  # change units to hours

    df_time_temp[!, "title"] = [L"Time [h]"]

    if nrow(df_time_iter) == 0
        df_time_iter = df_time_temp
    else
        df_time_iter = vcat(df_time_iter, df_time_temp)
    end
end

header_cols = ["name", names(df_run_simulations_iter)..., "title"]
df_list_rewards = setdiff(names(df_time_iter), ["id_run"; header_cols])

df_time_iter = hcat(df_run_simulations_iter[df_time_iter[!, "id_run"], :], df_time_iter[!, df_list_rewards])

CSV.write("comp_time.csv", df_time_iter)

df_time_iter

#### ITER_MODE: Create comparison of time per iteration

In [ ]:
df_history_iter = DataFrame()

for id_run in 1:n_iter
    df_history_temp = deepcopy(dict_iter[id_run]["df_history"])
    df_history_temp[!, "id_run"] .= id_run

    df_history_temp = df_history_temp[!, ["id_run"; setdiff(names(df_history_temp), ["id_run"])]]

    if nrow(df_history_iter) == 0
        df_history_iter = df_history_temp
    else
        df_history_iter = vcat(df_history_iter, df_history_temp)
    end
end

df_history_iter

In [ ]:
for ((k,), v) in pairs(groupby(df_history_iter, [:name]))
    # initialize plot
    p = plot()

    # loop over run ids
    for ((k_r,), v_r) in pairs(groupby(v, [:id_run]))
        plot!(v_r[!, :iteration], v_r[!, :elapsed_time], label="id_run: $k_r")
    end

    xlabel!("Iteration [#]")
    ylabel!("Elapsed time [s]")

    display(p)
end

#### ITER_MODE: Comparison of iteration time

In [ ]:
for ((k,), v) in pairs(groupby(df_history_iter, [:name]))
    # initialize plot
    p = plot()

    # loop over run ids
    for ((k_r,), v_r) in pairs(groupby(v, [:id_run]))
        delta_time = [v_r[1, :elapsed_time]; v_r[2:end, :elapsed_time] .- v_r[1:end-1, :elapsed_time]]
        plot!(v_r[!, :iteration], delta_time, label="id_run: $k_r")
    end

    xlabel!("Iteration [#]")
    ylabel!("Elapsed time [s]")

    display(p)
end

#### ITER_MODE: Comparison of upper and lower bounds

In [ ]:
rng_plot = 3

for ((k,), v) in pairs(groupby(df_history_iter, [:name]))
    # initialize plot
    # loop over run ids
    for ((k_r,), v_r) in pairs(groupby(v, [:id_run]))
        p = plot(v_r[!, :iteration], v_r[!, :value_min_surplus], label="Upper bound", color="red", title="$k - id_run: $k_r")
        plot!(v_r[!, :iteration], v_r[!, :lower_problem_min_surplus], label="Lower bound", color="blue")

        best_obj = minimum(v_r[!, :value_min_surplus])

        xlabel!("Iteration [#]")
        ylabel!("Obj. value")
        ylims!(p, -rng_plot*best_obj, rng_plot*best_obj)
    
        display(p)
    end
end